# Script to explore radar picks

## Import packages

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from haversine import haversine, Unit
import numpy as np
import pandas as pd


: 

## set basic parameters

In [2]:

# define cores

class core:
    def __init__(self, name, lat, lon, depth):
        self.name = name
        self.lat = lat
        self.lon = lon
        self.depth = depth

    def get_dist_to_track(self,lat,lon):

        self.track_dist=10000
        for i in range(len(lat)):
            d = haversine((lat[i],lon[i]),(self.lat,self.lon),unit=Unit.METERS)
            if d < self.track_dist:
                self.track_dist = d
                self.track_idx = i

ALHIC2302 = core('ALHIC2302',-76.74543,159.37513, 143)
ALHIC2201 = core('ALHIC2201',-76.732027,159.359551, 90)
ALHIC1902 = core('ALHIC1902',-76.745357,159.374108, 206.53)
ALHIC1901 = core('ALHIC1901',-76.732376,159.356125, 159.84)


## Calculate Weighted Percentile

In [3]:
# define function for weighted_percentile
def weighted_percentile(values, weights, percentiles):
    
    # Convert percentiles to fractions
    percentiles = [p / 100 for p in percentiles]

    # Convert values and weights to numpy arrays
    values = np.array(values)
    weights = np.array(weights)
    percentiles = np.array(percentiles)

    # Check for NaN values and remove them
    nan_mask = np.isnan(values) | np.isnan(weights)
    if np.any(nan_mask):
        print("NaN values found and will be removed.")
    values = values[~nan_mask]
    weights = weights[~nan_mask]
    
    # Sort values and weights by values
    sorted_indices = np.argsort(values)
    sorted_values = values[sorted_indices]
    sorted_weights = weights[sorted_indices]
    
    # Compute the cumulative sum of weights
    cumulative_weights = np.cumsum(sorted_weights)
    
    # Normalize the cumulative weights to get the cumulative distribution
    cumulative_distribution = cumulative_weights / cumulative_weights[-1]
    
    # Interpolate to find the weighted percentiles
    weighted_percentiles = np.interp(percentiles, cumulative_distribution, sorted_values)
    
    return weighted_percentiles

# define function to calculate percentiles from dataframe with true dip angles
def calc_percentiles(df,percentiles):

    # initialize list to store dip statistics
    dip_stats = []
    # define if we are looking at AC or DC (AC is default)
    ACorDC = 'AC'

    # loop through each row in the dataframe
    for index,row in df.iterrows():

        # pull out approriate data
        dip = np.array(row[ACorDC+'-true-angles'])
        scores = np.array(row[ACorDC+'-true-scores'])
        
        # plot dip
        a = weighted_percentile(dip, scores, percentiles)
       # weighted_percentile(dip, percentiles, weights=scores, interpolation='step')
        dip_stats.append(a)

        # check
        for i in range(len(a)-1):
            if a[i]>a[i+1]:
                print('Error: Percentiles are not in order in section '+row['section'])

    df['dip_percentiles']  = dip_stats
    return df

## Basic data import and exploration

In [ ]:
# Function to load the shapefile and check basic information
def load_shapefile(file_path):
    try:
        # Load the shapefile using geopandas
        gdf = gpd.read_file(file_path)
        
        # Display basic information about the data
        print("Loaded shapefile successfully!")
        print("Number of features:", len(gdf))
        print("CRS (Coordinate Reference System):", gdf.crs)
        print("First few rows of the data:")
        print(gdf.head())
        
        # Show the columns in the shapefile
        print("\nColumns in the shapefile:")
        print(gdf.columns)

        # You can access the geometry (the spatial component) and attributes here
        print("\nSample geometry:")
        print(gdf.geometry.head())

    except Exception as e:
        print(f"Error loading shapefile: {e}")

    return gdf

# Provide the path to the .shp file
#shapefile_path = "../../../radar_picks/picks-v2-culled-2melev.shp"
shapefile_path = "../../../radar_picks/picks-v2-culled.shp"
gdf = load_shapefile(shapefile_path)


In [ ]:
gpd.read_file(shapefile_path)

In [ ]:
# tool to find the correct lines

fig,ax = plt.subplots(figsize=(14,14))

gdf.plot(ax = ax, color='r')

lo = 1510 # this is right
hi = 1750 # this is right
line23 = gdf[lo:hi]
line23.plot(ax=ax, color='b')

lo = 0 # this is right
hi = 437 # this is right
line02 = gdf[lo:hi]
line02.plot(ax=ax, color='g')



In [ ]:
fig,ax = plt.subplots(2,1,figsize=(8,8))

for line,name,ax,cores,core_angle in zip([line23,line02],
                                         ['Track 23','Track 2'],
                                         [ax[0],ax[1]],
                                         [[ALHIC2302,ALHIC1902],[ALHIC2201,ALHIC1901]],
                                         [alhic2302_angle,alhic2201_angle]):
    # Load radar angle data
    angles = []
    angle_depth = []   
    for idx,row in core_angle.iterrows():
        percentiles = row['dip_percentiles']
        median = percentiles[2]
        angle_depth.append(row['depth'])
        angles.append(median)
    
    # calculate the mean angle for all sections where the IQR is less than 20 degrees
    valid_angles = []
    for idx, row in core_angle.iterrows():
        percentiles = row['dip_percentiles']
        iqr = abs(percentiles[3] - percentiles[1])
        if iqr < 20:
            valid_angles.append(percentiles[2])
    if valid_angles:
        angle_mean = np.mean(abs(valid_angles))
    else:
        print("ERROR: NO VALID ANGLES")



    # Get the latitude, longitude, and depth values
    lat = line['Lat()'].to_numpy()
    lon = line['Long()'].to_numpy()
    depth = line['Layer 1 De'].to_numpy()

    # find closest point on line
    for core in cores:
        core.get_dist_to_track(lat,lon)
        print(" Distance between "+core.name+" and line is ",core.track_dist)

    # compute distance along track
    dist = []
    for la,lo in zip(lat,lon):
        dist.append(haversine((lat[0],lon[0]),(la,lo),unit=Unit.METERS))

    # plot the data
    ax.plot(dist,depth,label='Bed Pick (Nestbit & Brook, 2023)')

    # plot core location
    color = ['r--','k--']
    for core,c in zip(cores,color):
        label = core.name + ' (~'+str(round(core.track_dist))+' m from track)'
        ax.plot([dist[core.track_idx],dist[core.track_idx]],[0,core.depth],c,label=label)



    # calculate the bed slope near the core
    core = cores[0] # pick relevant core
    idx = core.track_idx
    dist_min = dist[idx] - 20
    dist_max = dist[idx] + 20
    idx_min = int(round(np.interp(dist_min,dist,range(len(dist)))))
    idx_max = int(round(np.interp(dist_max,dist,range(len(dist)))))
    slope = abs((depth[idx_max] - depth[idx_min])/(dist[idx_max] - dist[idx_min]))
    dip = np.arctan(slope)*180/np.pi
    print('Bed slope near '+core.name+' is ',slope,' or ',dip,'°')

    # Plot ECM angle
    hor = (dist[idx_max] - dist[idx_min])/2
    if name == 'Track 23':
        mult = 1
    else:
        mult = -1
    offset = hor * np.tan(angle_mean*np.pi/180)
    ax.plot([dist[idx_min],dist[idx_max]],
            [angle_depth_mean-offset*mult,angle_depth_mean+offset*mult],
            'r',
            label = 'Average ECM Layer Dip ('+str(round(angle_mean,1))+'°)')


    # plot the bed slope
    ax.plot([dist[idx_min],dist[idx_max]],[depth[idx_min],depth[idx_max]],
            'g',
            label='Bed Slope under '+core.name+' ('+str(round(dip,1))+'°)',
            linewidth=4)

    # set axis to equal scale
    ax.axis('equal')

    # apply x axis limits
    if name == 'Track 23':
        ax.set_xlim(0,800)
        ax.legend(loc='upper right')
    else:
        ax.set_xlim(0,800)
        ax.legend(loc='lower right')


    # plot housekeeping
    title = name + ' - ' + cores[0].name +'/'+cores[1].name 
    ax.set_title(title)
    ax.set_xlabel('Distance along track (m)')
    ax.set_ylabel('Depth (m)')
    ax.set_ylim(300,0)
    ax.grid(True)

fig.tight_layout()